In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
#Add 'functions' folder to notebook directory
cwd = os.getcwd()
fn_dir = cwd + '/functions'
sys.path.append(fn_dir)
#Functions in 'functions' folder
import fn_sql
import fn_db

In [11]:
dfndb, db_connection = fn_db.liiondb()

---
### 1. OCV curves for cathodes with higher than 50% nickel content

In [32]:
QUERY = '''
        SELECT DISTINCT data.data_id,parameter.symbol,parameter.name as parameter, material.name as material,data.raw_data, parameter.units_output, paper.paper_tag, paper.doi
        FROM data
        JOIN paper ON paper.paper_id = data.paper_id
        JOIN material ON material.material_id = data.material_id
        JOIN parameter ON parameter.parameter_id = data.parameter_id
        WHERE parameter.name = 'half cell ocv'
        AND material.ni > 0.5
        '''
df = pd.read_sql(QUERY,dfndb)
df.head(5) #Print top 5 to save space

,data_id,symbol,parameter,material,raw_data,units_output,paper_tag,doi
0,129,${U_\text{eq}}$,half cell ocv,NMC811,"{{0.256747889,4.2935653},{0.25950522,4.2768621...",V,Chen2020,10.1149/1945-7111/ab9050
1,338,${U_\text{eq}}$,half cell ocv,NMC622,"{{0.0,4.4},{0.028439493,4.3054755039999995},{0...",V,Gao2018,10.1021/acs.jpclett.8b02229
2,422,${U_\text{eq}}$,half cell ocv,NCA,None,V,Kim2011,10.1149/1.3597614
3,460,${U_\text{eq}}$,half cell ocv,NCA,"{{0.3690799576451973,4.208233276157805},{0.380...",V,Albertus2009,10.1149/1.3129656
4,593,${U_\text{eq}}$,half cell ocv,NCA,"{{0.13,4.55},{0.26,4.35},{0.28,4.25},{0.34,4.1...",V,Abraham2008,10.1016/j.electacta.2007.09.018


---
### 2. Li diffusivities in graphite that are valid at 10 °C

In [30]:
QUERY = '''
        SELECT DISTINCT data.data_id,parameter.symbol,parameter.name as parameter, material.name as material,data.raw_data, parameter.units_output, data.temp_range, paper.paper_tag, paper.doi
        FROM data
        JOIN paper ON paper.paper_id = data.paper_id
        JOIN material ON material.material_id = data.material_id
        JOIN parameter ON parameter.parameter_id = data.parameter_id
        WHERE parameter.name = 'diffusion coefficient'
        AND material.class = 'negative'
        AND material.gr = 1
        AND 283 BETWEEN lower(data.temp_range) AND upper(data.temp_range)
        '''
df = pd.read_sql(QUERY,dfndb)
df.head(5) #Print top 5 to save space

,data_id,symbol,parameter,material,raw_data,units_output,temp_range,paper_tag,doi
0,69,$D$,diffusion coefficient,Graphite,"{{0.04291659469592768,0.0000000000002531898360...",m^2*s^-1,"[250, 333]",Ecker2015,10.1149/2.0551509jes
1,72,$D$,diffusion coefficient,Graphite,"{{0.14962963,0.000000000000041723300000000003}...",m^2*s^-1,"[250, 333]",Ecker2015,10.1149/2.0551509jes
2,74,$D$,diffusion coefficient,Graphite,"{{0.033255915,0.000000000000109},{0.149546849,...",m^2*s^-1,"[250, 313]",Schmalstieg2018,10.1149/2.0321816jes
3,663,$D$,diffusion coefficient,Graphite,"{{0.0,0.0000000000001},{0.1,0.00000000000001},...",m^2*s^-1,"[273, 298]",Cabanero2018,10.1149/2.0301805jes
4,938,$D$,diffusion coefficient,Graphite,0.000000000000001,m^2*s^-1,"[283, 283]",Mastali2016,10.1149/2.1151613jes


---
### 3. See all separator porosities

In [35]:
QUERY = '''
        SELECT DISTINCT data.data_id,parameter.name, material.name, paper.paper_tag,data.raw_data
        FROM data
        JOIN paper ON paper.paper_id = data.paper_id
        JOIN material ON material.material_id = data.material_id
        JOIN parameter ON parameter.parameter_id = data.parameter_id
        WHERE parameter.name = 'porosity'
        AND material.class = 'separator'
        '''
df = pd.read_sql(QUERY,dfndb)
df.head(5) #Print top 5 to save space

,data_id,name,name,paper_tag,raw_data
0,21,porosity,Polymer Separator,Ecker2015,0.508
1,65,porosity,Polymer Separator,Schmalstieg2018,0.395
2,151,porosity,Polymer Separator,Chen2020,0.47
3,469,porosity,Polymer Separator,Albertus2009,0.37
4,504,porosity,Polymer Separator,Doyle2003,0.5


---
### 4. All papers that publish parameters on LFP


In [44]:
QUERY = '''
        SELECT DISTINCT paper.paper_tag, paper.title, paper.doi
        FROM paper
        JOIN data ON data.paper_id = paper.paper_id
        JOIN material ON data.material_id = material.material_id
        WHERE material.name = 'LFP'
        '''
df = pd.read_sql(QUERY,dfndb)
df.head(5) #Print top 5 to save space

,paper_tag,title,doi
0,Cabanero2018,Direct Determination of Diffusion Coefficients...,10.1149/2.0301805jes
1,Delacourt2011,Analysis of lithium deinsertion/insertion in {...,10.1016/j.electacta.2011.03.030
2,Farkhondeh2014,Full-Range Simulation of a Commercial {LiFePO}...,10.1149/2.094401jes
3,Han2004,Electrochemical modeling of intercalation proc...,10.1016/j.electacta.2004.05.024
4,Heubner2015,Investigation of charge transfer kinetics of L...,10.1016/j.jpowsour.2015.04.103


---
### 5. Parameters that have been measured with EIS


In [45]:
QUERY = '''
        SELECT DISTINCT parameter.name
        FROM parameter
        JOIN data ON data.parameter_id = parameter.parameter_id
        JOIN data_method ON data.data_id = data_method.data_id
        JOIN method ON data_method.method_id = method.method_id
        WHERE  method.name = 'EIS'
        '''
df = pd.read_sql(QUERY,dfndb)
df

,name
0,tortuosity
1,liquid phase transport efficiency
2,exchange current density
3,porosity
4,reaction rate constant
5,ionic conductivity
6,diffusion coefficient


---
### 6. Full electrolyte parameterizations


In [47]:
QUERY = '''
        SELECT DISTINCT material.name, paper.paper_tag, parameter.name as param_name
        FROM material
        JOIN data ON data.material_id = material.material_id
        JOIN paper ON data.paper_id = paper.paper_id
        JOIN parameter ON data.parameter_id = parameter.parameter_id
        WHERE material.class = 'electrolyte'
        AND parameter.name IN ('ionic conductivity','diffusion coefficient','transference number','thermodynamic factor')
        '''
df = pd.read_sql(QUERY,dfndb)
df.head(5) #Print top 5 to save space
# df

,name,paper_tag,param_name
0,LiPF6:EC:DMC 1:1,Farkhondeh2014,transference number
1,LiPF6:PC,Stewart2008,thermodynamic factor
2,LiPF6:EC:EMC 3:7,Nyman2008,transference number
3,LiPF6:EMC,Logan2018,ionic conductivity
4,LiTFSI:ACN,Lundgren2015,diffusion coefficient


---
### 7. See the Doyle 1996 paper parameters


In [50]:
QUERY = '''
        SELECT DISTINCT data.data_id,parameter.name, material.name, paper.paper_tag,data.raw_data, parameter.units_output, data.notes
        FROM data
        JOIN paper ON paper.paper_id = data.paper_id
        JOIN material ON material.material_id = data.material_id
        JOIN parameter ON parameter.parameter_id = data.parameter_id
        WHERE paper.paper_tag = 'Doyle1996'
        '''
df = pd.read_sql(QUERY,dfndb)
df.head(5) #Print top 5 to save space
# df

,data_id,name,name,paper_tag,raw_data,units_output,notes
0,296,diffusion coefficient,LMO,Doyle1996,0.0000000000001,m^2*s^-1,Assumed from literature in ref 9 from Doyle1996
1,297,electronic conductivity,LMO,Doyle1996,3.8,S*m^-1,Scaled using Bruggeman correction in particle ...
2,298,exchange current density,LMO,Doyle1996,0.8,A*m^-2,Assumed magnitude from typical intercalation k...
3,299,maximum concentration,LMO,Doyle1996,22860,mol*m^-3,Calculated from crystal density
4,300,particle radius,LMO,Doyle1996,0.0000085,m,Optimized from matching experimental data
